In [ ]:
import os
import pandas as pd

In [ ]:
file_P = os.path.join(os.getcwd(), 'Elspotprices2nd.csv')
df_prices = pd.read_csv(file_P)
df_prices['HourUTC'] = pd.to_datetime(df_prices['HourUTC'])

file_P = os.path.join(os.getcwd(), 'ProdConData.csv')
df_data = pd.read_csv(file_P)
df_data['HourUTC'] = pd.to_datetime(df_data['HourUTC'])


1.1

Use NO exogenous variables in your model and *make day-ahead prediction for your testing dataset*.

You can use a *seasonal ARIMA* or FourierFeaturizer and any data transformation you want in your model, but *no exogenous features* from df data. 

*Establish a suitable persistence forecast* and *report the RMSE values in both cases (your model and persistence)*. 

Briefly discuss your results.

1.2

Add any exogenous variables you want (maximum 3) and repeat the process (choose/optimize
your model and evaluate it for the day-ahead prediction).

What exogenous variables helped you improve the prediction and how did you choose the specific ones?

Report the RMSE value and compare your results with those from task 1.1 and briefly discuss them